In [1]:
from B_field_sources import create_coil, magpy
from ion_model import PolarVector
import numpy as np
import plotly.graph_objs as go

# external_field = create_coil(1, 1000, 0.1, current=-20)

coilx = create_coil(20, 100, 0.1)
coily = coilx.copy(position=(0, 300, 0))
coil_main = create_coil(20, 100, 0.1, current=1.9)
coil_main.move((300, 0, 0))
# coilx.copy(position=(-300, 0, 0))

coilx.move((-300, 0, 0))

coily.rotate_from_angax(90, 'x', start=0)
coilx.rotate_from_angax(90, 'y', start=0)
coil_main.rotate_from_angax(-90, 'y', start=0)

coilz = create_coil(20, 300, 0.1, current=-1)
coilz.move((0, 0, -150))

all_coils = coilx + coily + coil_main + coilz #+ external_field

# helmholtz.rotate_from_angax(90, 'x', start=0)

# create grid
ts = np.linspace(-400, 400, 15)
grid = np.array([[(x,0,z) for x in ts] for z in ts])

uniform_vector = PolarVector(0, 0, np.pi/2)

# compute and plot field of coil1
# B = magpy.getB(all_coils, grid) + uf
# B_vec = magpy.getB(all_coils, (0, 0, 0)) * 20000
# Bamp = np.linalg.norm(B, axis=2)
# Bamp /= np.amax(Bamp)

fig = magpy.show(*all_coils, return_fig=True, backend="plotly")

grid = np.array([[[(x,y,z) for x in ts] for y in ts] for z in ts])
X, Y, Z = grid[:, :, :, 0], grid[:, :, :, 1], grid[:, :, :, 2]
vec = uniform_vector.v[:, 1].astype(np.float64)
uf = np.zeros(grid.shape) + vec
print(uf.shape, vec.shape)
B = magpy.getB(all_coils, grid)
B += uf


Bx, By, Bz = B[:, :, :, 0], B[:, :, :, 1], B[:, :, :, 2]

grad = np.gradient(B)
dell = grad[0] + grad[1] + grad[2] + grad[3]

dellx, delly, dellz = dell[:, :, :, 0], dell[:, :, :, 1], dell[:, :, :, 2]

dell = dellx + delly + dellz

minx, miny, minz = np.unravel_index(np.argmin(dell, axis=None), dell.shape)

x, y, z, Bx, By, Bz = X.flatten(), Y.flatten(), Z.flatten(), Bx.flatten(), By.flatten(), Bz.flatten()

x, y, z, Bx, By, Bz = x.tolist(), y.tolist(), z.tolist(), Bx.tolist(), By.tolist(), Bz.tolist()

spread = 200
num_start = 4
start_x = np.linspace(minx-spread, minx+spread, num_start)
start_y = np.linspace(miny-spread, miny+spread, num_start)
start_z = np.linspace(minz-spread, minz+spread, num_start)

SX, SY, SZ = np.meshgrid(start_x, start_y, start_z)
start_x = SX.flatten()
start_y = SY.flatten()
start_z = SZ.flatten()


data_plot = [go.Streamtube(
    x=x, y=y, z=z, u=Bx, v=By, w=Bz,
    sizeref = 0.3,
    starts=dict(
        x = start_x,
        y = start_y,
        z = start_z),
    colorscale = 'jet',
    showscale = True,
    maxdisplayed = 700,
    name="B fields"
    )]


# Create your layout with a button
updatemenus = [
    {
        "buttons": [
            {
                "args": [
                    {"visible": [True, False]}, # toggle Trace 2
                    {"title": "B field"}
                ],
                "label": "B fields",
                "method": "update",
            },
        ],
        "type": "buttons",
        "showactive": True,
        "x": 0.05,
        "y": 1.05,
    }
]


# vector_field = go.Cone(x=x, y=y, z=z, u=Bx, v=By, w=Bz, sizemode="absolute", sizeref=5, colorscale='Blues')
# fig.add_trace(vector_field)
fig.add_trace(data_plot[0])
fig.update_layout(showlegend=False, coloraxis_colorbar=dict(title="[mT]"), updatemenus=updatemenus)
fig.show()

(15, 15, 15, 3) (3,)
